In [13]:
using Documenter;
using Crayons;
using Logging;
using JuMP, MathOptInterface;
const MOI = MathOptInterface;
const MOIU = MOI.Utilities;

$$
\begin{array}{rl}
    \max & \mathbf{c}^{\small\mathbf{\text{T}}}\mathbf{x} \\
    \text{s.t.} & \mathbf{w}^{\small\mathbf{\text{T}}}\mathbf{x} +\mathbf{s} = C\\
    ~ & \mathbf{x}_i \in \{0, 1\}
\end{array}
$$

In [22]:
# Model definition
# References:
# [1] https://jump.dev/MathOptInterface.jl/stable/tutorials/example/

model = MOIU.Model{Float64}()

n = 3;
m = 32;
c = [1.0, 2.0, 3.0];
w = [3, 5, 10.];
C = 32.0; #3.2

x = MOI.add_variables(model, n);
s = MOI.add_variables(model, m);

# ---------
# Objective
# ---------
MOI.set(model, MOI.ObjectiveSense(), MOI.MAX_SENSE)

MOI.set(
   model,
   MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
   MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0),
);

# -----------
# Constraints
# -----------
MOI.add_constraint(
   model,
   MOI.ScalarAffineFunction([MOI.ScalarAffineTerm.(w, x); MOI.ScalarAffineTerm.(ones(32), s)], 0.0),
   MOI.EqualTo(C),
);

for xᵢ in x
   MOI.add_constraint(model, xᵢ, MOI.ZeroOne())
end

for sᵢ in s
   MOI.add_constraint(model, sᵢ, MOI.ZeroOne())
end

In [126]:
module ToQUBO
    # -*- ToQUBO.jl -*-
    using Crayons, Documenter, Logging
    using JuMP, MathOptInterface
    const MOI = MathOptInterface
    const MOIU = MathOptInterface.Utilities
    const MOIB = MathOptInterface.Bridges

    const SVF = MOI.SingleVariable
    const VVF = MOI.VectorOfVariables
    const SAF{T} = MOI.ScalarAffineFunction{T}
    const VAF{T} = MOI.VectorAffineFunction{T}
    const SQF{T} = MOI.ScalarQuadraticFunction{T}
    const VQF{T} = MOI.VectorQuadraticFunction{T}

    const EQ{T} = MOI.EqualTo{T}
    const LT{T} = MOI.LessThan{T}
    const GT{T} = MOI.GreaterThan{T}

    const VI = MOI.VariableIndex
    const CI = MOI.ConstraintIndex

    const ZO = MOI.ZeroOne

    include("../src/posiform.jl")
    include("../src/supported.jl")
    include("../src/moiwrapper.jl")

    function subscript(v::VI)
        return subscript(v.value)
    end


    function penalty(p::Posiform{S, T}) where {S, T}
        return sum(abs(v) for (k, v) in p if !isempty(k))
    end
    
    """
    """
    function toqubo(model::MOI.ModelLike; quantum::Bool=false)
    
        # Support Validation
        supported_objective(model)
        supported_constraints(model)

        return __toqubo(model, quantum)
    end;

    """
    """
    function __toqubo(model::MOI.ModelLike, quantum::Bool)

        qubo_model = __toqubo(model)

        if quantum
            # Objective Sense
            sense = MOI.get(qubo_model, MOI.ObjectiveSense())
            if sense === MOI.MAX_SENSE
                # TODO: Invert Objective Function Signal

                MOI.set(qubo_model, MOI.ObjectiveSense(), MOI.MIN_SENSE)
            elseif sense === MOI.MIN_SENSE
                # Ok!
            else
                error("Invalid Objective Sense '$sense'")
            end
        end

        return qubo_model
    end

    """
    """
    function __toqubo(model::MOI.ModelLike)

        sense = MOI.get(model, MOI.ObjectiveSense())

        T = Float64

        p = Posiform{VI, T}()
        ∅ = Set{VI}()

        # Store Binary and Non-Binary Variables
        v = Set{VI}() # B
        x = Set{VI}() # NB

        for cᵢ in MOI.get(model, MOI.ListOfConstraintIndices{VI, ZO}())
            push!(v, MOI.get(model, MOI.ConstraintFunction(), cᵢ))
        end

        println(crayon"#af00af", "Binary variables:", crayon"#ffffff")
        if isempty(v)
            println("∅")
        else
            println(join(["v$(subscript(vᵢ))" for vᵢ in v], ", "))
        end

        println(crayon"#00afaf", "Non-Binary variables:", crayon"#ffffff")
        if isempty(x)
            println("∅")
        else
            println(join(["x$(subscript(xᵢ))" for xᵢ in x], ", "))
        end
        
        # Objective
        F = MOI.get(model, MOI.ObjectiveFunctionType())
        f = MOI.get(model, MOI.ObjectiveFunction{F}())

        if F === SAF{T}
            p += f.constant
            for tᵢ in f.terms
                cᵢ = tᵢ.coefficient
                vᵢ = tᵢ.variable
                if vᵢ in v
                    p[vᵢ] += cᵢ
                else
                    @warn "Variable 'v$(subscript(vᵢ))' is not binary and may need expansion"
                end
            end
        else
            @warn "I Don't know how to deal with objective functions of type '$F'"
        end

        ρ = penalty(p)

        # Constraints
        for (F, S) in MOI.get(model, MOI.ListOfConstraints())
            if F === VI
                if S === ZO
                    continue # These were already accounted for...
                elseif S === EQ{T}
                    # Fixed Variable!
                    @warn "There are Fixed variables!"
                    
                end
            elseif F === SAF{T}
                if S === EQ{T}
                    for cᵢ in MOI.get(model, MOI.ListOfConstraintIndices{F, S}())
                        qᵢ = Posiform{VI, T}()

                        Aᵢ = MOI.get(model, MOI.ConstraintFunction(), cᵢ)
                        bᵢ = MOI.get(model, MOI.ConstraintSet(), cᵢ).value

                        for aⱼ in Aᵢ.terms
                            cⱼ = aⱼ.coefficient
                            vⱼ = aⱼ.variable

                            if vⱼ in v
                                qᵢ[vⱼ] += cⱼ
                            else
                                error("Non-Binary variable '$(vᵢ)' needs expansion")
                            end
                        end

                        if sense === MOI.MAX_SENSE
                            p -= ρ * (qᵢ - bᵢ) ^ 2
                        elseif sense === MOI.MIN_SENSE
                            p += ρ * (qᵢ - bᵢ) ^ 2
                        else
                            error("Unknown Objective sense $(sense)")
                        end
                    end
                end
            else
                error("Unkown Constraint Type $F")
            end
        end

        return p
    end
end

p = ToQUBO.toqubo(model)

function toqubo(p::Posiform{S, T}) where {S, T}
    
end

Binary variables:
v₅, v₁₆, v₂₀, v₃₅, v₁₂, v₂₄, v₂₈, v₈, v₁₇, v₃₀, v₁, v₁₉, v₂₂, v₂₃, v₆, v₃₂, v₁₁, v₉, v₃₁, v₁₄, v₃, v₂₉, v₇, v₂₅, v₃₃, v₃₄, v₄, v₁₃, v₁₅, v₂, v₁₀, v₁₈, v₂₁, v₂₆, v₂₇
Non-Binary variables:
∅


-12.0 x₃₄ x₃₅ - 12.0 x₃₅ x₇ - 12.0 x₂₆ x₁₂ - 12.0 x₁₅ x₁₀ - 12.0 x₂₀ x₃₁ - 12.0 x₉ x₈ - 12.0 x₂₄ x₈ - 12.0 x₁₈ x₃₁ - 12.0 x₄ x₂₄ - 12.0 x₁₃ x₂₀ - 36.0 x₁ x₁₉ - 12.0 x₂₁ x₁₁ - 12.0 x₃₄ x₂₉ - 12.0 x₅ x₁₄ - 12.0 x₃₀ x₁₉ + 378.0 x₄ - 12.0 x₅ x₁₆ - 60.0 x₆ x₂ - 60.0 x₂ x₂₆ - 12.0 x₅ x₂₃ - 36.0 x₁₄ x₁ - 12.0 x₂₉ x₁₆ - 12.0 x₂₈ x₁₉ - 120.0 x₁₇ x₃ - 12.0 x₅ x₁₈ - 12.0 x₃₄ x₁₄ - 12.0 x₄ x₂₃ + 378.0 x₁₈ - 12.0 x₂₂ x₆ - 12.0 x₆ x₂₇ + 378.0 x₁₀ - 12.0 x₂₇ x₈ - 12.0 x₃₃ x₃₀ - 12.0 x₂₀ x₁₄ - 120.0 x₂₁ x₃ - 12.0 x₁₆ x₁₇ - 12.0 x₃₄ x₂₆ - 12.0 x₂₇ x₁₂ - 12.0 x₁₁ x₁₉ - 12.0 x₁₆ x₂₄ - 12.0 x₁₂ x₂₃ - 12.0 x₂₀ x₂₈ - 12.0 x₂₂ x₁₈ - 12.0 x₃₄ x₂₇ - 12.0 x₃₄ x₁₆ - 12.0 x₃₂ x₄ - 12.0 x₁₃ x₂₈ - 12.0 x₄ x₂₉ - 12.0 x₃₂ x₁₅ - 12.0 x₈ x₁₇ - 12.0 x₂₁ x₂₆ - 12.0 x₆ x₁₂ - 12.0 x₂₂ x₁₂ - 12.0 x₂₆ x₂₄ - 12.0 x₆ x₂₀ - 12.0 x₂₇ x₉ - 12.0 x₁₆ x₁₁ - 12.0 x₁₅ x₃₀ - 12.0 x₂₉ x₁₄ - 12.0 x₂₉ x₃₀ - 120.0 x₂₇ x₃ - 12.0 x₂₂ x₂₁ - 120.0 x₃ x₁₉ - 12.0 x₁₃ x₁₇ - 12.0 x₇ x₁₉ - 12.0 x₆ x₇ - 120.0 x₂₈ x₃ - 12.0 x₂₂ x₁₆ - 12.0 x₂₅ x₂₃ - 1

In [127]:
p.degree

2